<a href="https://colab.research.google.com/github/must19y/GPT-Transformer-model-from-Scratch/blob/main/GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from  tensorflow.keras import layers,metrics,losses,models

In [ ]:
from google.colab import files
files.upload()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
!kaggle datasets download -d zynicide/wine-reviews


Dataset URL: https://www.kaggle.com/datasets/zynicide/wine-reviews
License(s): CC-BY-NC-SA-4.0
100% 50.9M/50.9M [00:03<00:00, 26.5MB/s]
100% 50.9M/50.9M [00:03<00:00, 15.0MB/s]


In [ ]:
!unzip wine-reviews.zip -d wine-reviews


Archive:  wine-reviews.zip
  inflating: wine-reviews/winemag-data-130k-v2.csv  
  inflating: wine-reviews/winemag-data-130k-v2.json  
  inflating: wine-reviews/winemag-data_first150k.csv  


In [ ]:
import pandas as pd

# Load the dataset using pandas
wine_data = pd.read_csv('/content/wine-reviews/winemag-data_first150k.csv')

print(wine_data.columns)

# Filter data: here we're assuming the columns 'title' and 'description' exist
filtered_data = [
    'Review for' + x['description'] for _,x in wine_data.iterrows()
    if 'description' in x and x['description'] is not None
]

# Display an example of a filtered review
print(filtered_data[0])


Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'variety', 'winery'],
      dtype='object')
Review forThis tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022–2030.


In [ ]:
import re
import string

def pad_punctuation(s):
  s = re.sub(f"([{string.punctuation}])", r' \1 ', s)
  s = re.sub(' +', ' ', s)
  return s

dataset=  [pad_punctuation(x) for x in filtered_data]


text_ds= tf.data.Dataset.from_tensor_slices(dataset).batch(128).shuffle(1000)

vectorize_layer= layers.TextVectorization(
    standardize = 'lower',
    max_tokens = 20000,
    output_mode = "int",
    output_sequence_length = 80 + 1,

)

vectorize_layer.adapt(text_ds)

vocab= vectorize_layer.get_vocabulary()




In [ ]:
## Now we need to creat the training data set we do this by initializing to x and y respectively

# Assuming  set up a TextVectorization layer as vectorize_layer
def prepare_wine_inputs(text):
    # Expand dimensions to match the expected input shape
    text = tf.expand_dims(text, -1)

    # Tokenize the text using the vectorize_layer
    tokenized_sentences = vectorize_layer(text)

    # Prepare the input (x) and target (y) by shifting
    x = tokenized_sentences[:, :-1]  # All tokens except the last one
    y = tokenized_sentences[:, 1:]   # All tokens except the first one (shifted by 1)

    return x, y

# Apply the function to the dataset (assuming text_ds is the  dataset containing wine descriptions)
train_ds = text_ds.map(prepare_wine_inputs)






In [ ]:
## we will use an attention mechanism which outputs a vector with attention scores of the embeddings concatenated.
## These scores are multiplied by a weight matrix and produced as an output . The embeddings calculate  the attention scores of the words till it self
import numpy as np

def causal_attention_mask(batch_size, n_dest, n_src, dtype):
   i = tf.range(n_dest)[:, None]
   j = tf.range(n_src)
   m = i >= j - n_src + n_dest
   mask = tf.cast(m, dtype)
   mask = tf.reshape(mask, [1, n_dest, n_src])
   mult = tf.concat(
   [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0)
   return tf.tile(mask, mult)
np.transpose(causal_attention_mask(1, 10, 10, dtype = tf.int32)[0])

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)

In [ ]:
## Now we need to create a model that calculates the attention scores and then implements the skip connections too using normalization layers and also the
## output from the attention multiheads

## The transformer model

class TransformerBlock(layers.Layer):
  def __init__(self, num_heads, key_dim, embed_dim, ff_dim, dropout_rate=0.1):
     super(TransformerBlock, self).__init__()
     self.num_heads = num_heads
     self.key_dim = key_dim
     self.embed_dim = embed_dim
     self.ff_dim = ff_dim
     self.dropout_rate = dropout_rate
     self.attn = layers.MultiHeadAttention(
     num_heads, key_dim, output_shape = embed_dim
     )
     self.dropout_1 = layers.Dropout(self.dropout_rate)
     self.ln_1 = layers.LayerNormalization(epsilon=1e-6)
     self.ffn_1 = layers.Dense(self.ff_dim, activation="relu")
     self.ffn_2 = layers.Dense(self.embed_dim)
     self.dropout_2 = layers.Dropout(self.dropout_rate)
     self.ln_2 = layers.LayerNormalization(epsilon=1e-6)
  def call(self, inputs):
    input_shape = tf.shape(inputs)
    batch_size = input_shape[0]
    seq_len = input_shape[1]
    causal_mask = causal_attention_mask(
    batch_size, seq_len, seq_len, tf.bool
    )
    attention_output, attention_scores = self.attn(
       inputs,
       inputs,
       attention_mask=causal_mask,
       return_attention_scores=True
    )
    attention_output = self.dropout_1(attention_output)
    out1 = self.ln_1(inputs + attention_output)
    ffn_1 = self.ffn_1(out1)
    ffn_2 = self.ffn_2(ffn_1)
    ffn_output = self.dropout_2(ffn_2)
    return (self.ln_2(out1 + ffn_output), attention_scores)













In [ ]:
## Now make the layer inputs with token and positional embeddings so that we can utilize the token+position embedding and keep track of the position of
##the sequence

class TokenAndPositionEmbedding(layers.Layer):
  def __init__(self, maxlen, vocab_size, embed_dim):
    super(TokenAndPositionEmbedding, self).__init__()
    self.maxlen = maxlen
    self.vocab_size =vocab_size
    self.embed_dim = embed_dim
    self.token_emb = layers.Embedding(
    input_dim=vocab_size, output_dim=embed_dim
    )
    self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
  def call(self, x):
   maxlen = tf.shape(x)[-1]
   positions = tf.range(start=0, limit=maxlen, delta=1)
   positions = self.pos_emb(positions)
   x = self.token_emb(x)
   return x + positions








In [ ]:
## Now training our GPT model

## GPT Model


MAX_LEN = 80
VOCAB_SIZE = 20000
EMBEDDING_DIM = 256
N_HEADS = 2
KEY_DIM = 256
FEED_FORWARD_DIM = 256
inputs = layers.Input(shape=(None,), dtype=tf.int32)
x = TokenAndPositionEmbedding(MAX_LEN, VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x, attention_scores = TransformerBlock(
N_HEADS, KEY_DIM, EMBEDDING_DIM, FEED_FORWARD_DIM
)(x)
outputs = layers.Dense(VOCAB_SIZE, activation = 'softmax')(x)
gpt = models.Model(inputs=inputs, outputs=[outputs, attention_scores])
gpt.compile("adam", loss=[losses.SparseCategoricalCrossentropy(), None])
gpt.fit(train_ds, epochs=1)



1180/1180 ━━━━━━━━━━━━━━━━━━━━ 19175s 16s/step - loss: 3.0081


In [ ]:
## finally we generate the text using a custom made class

from tensorflow.keras import callbacks

class TextGenerator(callbacks.Callback):
    def __init__(self,model,index_to_word, top_k=10):
        self.gpt_model= model
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }

    def sample_from(self, probs, temperature):
        probs= probs.flatten()
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y = self.gpt_model.predict(x)
            sample_token, probs = self.sample_from(y[0][-1], temperature)
            if sample_token >= len(self.index_to_word):
                sample_token = 0  # Use 0 for padding or unknown token

            info.append({'prompt': start_prompt, 'word_probs': probs})
            start_tokens.append(sample_token)
            start_prompt = start_prompt + ' ' + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("wine review", max_tokens=100, temperature=1.0)









In [ ]:
text_generator = TextGenerator(model=gpt,index_to_word=vocab)
generated_text = text_generator.generate(start_prompt="wine review for", max_tokens=100, temperature=1.0)
print(generated_text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step

generated text:
wine review for like with balanced 

[{'prompt': 'wine review for', 'word_probs': array([2.81588291e-04, 4.33853362e-03, 3.37696895e-02, ...,
       4.85280168e-07, 1.02884954e-07, 8.18544876e-08], dtype=float32)}, {'prompt': 'wine review for like', 'word_probs': array([2.1119135e-04, 3.2539051e-03, 2.5327282e-02, ..., 3.9183669e-07,
       6.6811594e-08, 7.9774111e-08], dtype=float32)}, {'prompt': 'wine review for like with', 'word_probs': array([1.6895309e-04, 2.6031244e-03, 2.0261828e-02, ..., 1.0336930e-07,
       4.7233122e-08, 1.1471105e-08], dtype=float32)}, {'prompt': 'wine review for like with balanced', 'word_probs': array([1.4079423e-04, 2.1692701e-03, 1.6884854e-02, ..., 1.5367401e-07,
       2.3308692e-08, 6.8190587e-09], dtype=float32)}]
